# Методы исследования характеристических свойств нейронных сетей с применением теоретико-игрового подхода

- **Теория**: И.В.Томилов ivan-tomilov3@yandex.ru
- **Реализация**: М.А.Зайцева maria@tail-call.ru
- **Поддержка CUDA**: А.Е.Григорьева admin@linkennt.ru
- **Ревизия**: 8

- **Другие ревизии**: [ссылка](https://disk.yandex.ru/d/aZozDpBlzh_z1A)
<!-- please do not append text into this block -->

- [ ] Добавить 2x больше эпох на датасетах, где был плохой перфоманс
- [ ] Примешивать к целевой переменной на этапе обучения шум с амплитудой ~0,03 дисперсии
- [ ] Третий пункт был связан со стабильными распределениями и варьированием ɑ от 1 до 2 — этот стабильный шум надо тоже к целевой переменной примешивать

## Setup

In [1]:
from cgtnnlib.constants import LEARNING_RATE, RANDOM_STATE
import cgtnnlib.training as tr
import cgtnnlib.datasets as ds
from cgtnnlib.ExperimentParameters import ExperimentParameters
from cgtnnlib.NoiseGenerator import target_dispersion_scaled_noise


iterations = 10
epochs = 10
dataset = ds.datasets[2]
noise_generator = target_dispersion_scaled_noise(
    dataset=dataset,
    factor=0.03,
    random_seed=RANDOM_STATE + 1,
)
p = 0.9


TORCH_DEVICE is cpu
target_dispersion_scaled_noise(Dataset #3 {
  name: "StudentPerformanceFactors"
  learning_task: LearningTask(name='regression', criterion=MSELoss(), dtype=torch.float32)
  classes_count: 1
  target: "Exam_Score"
  _data: DatasetData(df=      Hours_Studied  Attendance  Parental_Involvement  Access_to_Resources  \
0                23          84                    -1                    1   
1                19          64                    -1                    0   
2                24          98                     0                    0   
3                29          89                    -1                    0   
4                19          92                     0                    0   
...             ...         ...                   ...                  ...   
6602             25          69                     1                    0   
6603             23          76                     1                    0   
6604             20          90          

## Training

`datasets[2]` takes ~2m 30s to train 10 iterations

In [ ]:

for iteration in range(iterations):
    experiment_params = ExperimentParameters(iteration, p=p)

    tr.create_and_train_model(
        dataset,
        epochs,
        learning_rate=LEARNING_RATE,
        dry_run=False,
        p=p,
        iteration=iteration,
        noise_generator=noise_generator
    )

## Evaluation

In [2]:

from cgtnnlib.EvaluationParameters import EvaluationParameters
from cgtnnlib.Report import Report
from cgtnnlib.constants import LEARNING_RATE, RANDOM_STATE
from cgtnnlib.nn.AugmentedReLUNetwork import AugmentedReLUNetwork
import cgtnnlib.evaluate as ev
import cgtnnlib.datasets as ds
import cgtnnlib.path as ph
from cgtnnlib.ExperimentParameters import ExperimentParameters
from cgtnnlib.NoiseGenerator import target_dispersion_scaled_noise


iterations = 10
epochs = 10
dataset = ds.datasets[2]
p = 0.9
for iteration in range(iterations):
    experiment_params = ExperimentParameters(iteration, p=p)
    
    eval_params = EvaluationParameters(
        dataset=dataset,
        model_path=ph.model_path(
            dataset_number=dataset.number,
            model_type=AugmentedReLUNetwork,
            p=p,
            iteration=iteration,
            noise_generator=noise_generator,
        ),
        experiment_parameters=experiment_params,
        report_key=ph.eval_report_key(
            model_name=AugmentedReLUNetwork.__name__,
            dataset_number=dataset.number,
            p=p,
            iteration=iteration,
        )
    )
    
    report_name = ph.model_name(
        dataset_number=dataset.number,
        model_type=AugmentedReLUNetwork,
        p=p,
        iteration=iteration,
        noise_generator=noise_generator
    )
    report = Report(
        dir='pth/',
        filename=report_name + '.json',
        must_exist=True,
    )
    
    ev.eval_inner(
        eval_params=eval_params,
        constructor=AugmentedReLUNetwork,
        report=report
    )
    
    report.save()

    # tr.create_and_train_model(
    #     dataset,
    #     epochs,
    #     learning_rate=LEARNING_RATE,
    #     dry_run=False,
    #     p=p,
    #     iteration=iteration,
    #     noise_generator=target_dispersion_scaled_noise(
    #         dataset=dataset,
    #         factor=0.03,
    #         random_seed=RANDOM_STATE + 1,
    #     )
    # )

Evaluating model at pth/cgtnn-3Y-AugmentedReLUNetwork-gTDS3-P0.9_N9.pth...
Report saved to pth/cgtnn-3Y-AugmentedReLUNetwork-gTDS3-P0.9_N9.json.


## Analysis